In [ ]:
import os
os.environ['LC_ALL'] = 'en_US.UTF-8'
os.environ['LANG'] = 'en_US.UTF-8'
os.environ['LC_CTYPE'] = 'en_US.UTF-8'

In [ ]:
from pathlib import Path
from typing import Optional

def create_directory(path: Optional[Path] = None, dir_name: str = "output"):
    """
    Creates a directory at the specified path with the given directory name.
    If no path is provided, the current working directory is used.

    Parameters:
    - path (Optional[Path]): The path where the directory is to be created.
    - dir_name (str): The name of the directory to create.

    Returns:
    - Path object representing the path to the created directory.
    """
    # Use the current working directory if no path is provided
    working_dir = path if path is not None else Path('./')

    # Define the output directory path by joining paths
    output_directory = working_dir / dir_name

    # Create the directory if it doesn't exist
    output_directory.mkdir(parents=True, exist_ok=True)

    return output_directory

output_dir = create_directory(dir_name="fine-tuned-checkpoints")
print(f"Directory created at: {output_dir}")

Directory created at: fine-tuned-checkpoints


In [ ]:
%%capture
!pip install -q -U bitsandbytes
!pip install -q -U transformers
!pip install -q -U peft
!pip install -q -U accelerate
!pip install -q -U datasets
!pip install -q -U trl
!pip install ninja

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from trl import SFTTrainer
import torch

# Load model

In [ ]:
model_name = "Deci/DeciLM-7B"

gpu_memory = torch.cuda.get_device_properties(0).total_memory

do_quantization = gpu_memory < 20e9

if do_quantization:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit = True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        # bfloat works only on A100 (or ampere supported chip)
        bnb_4bit_compute_dtype=torch.bfloat16,
        # if you're using a T4 or non-ampere chip comment out the above and run this instead:
        # bnb_4bit_compute_dtype=torch.float16
    )

    decilm = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto",
        use_cache=True,
        trust_remote_code=True
    )
else:
    decilm = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        use_cache=True,
        trust_remote_code=True
    )


tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

tokenizer.pad_token = tokenizer.eos_token

tokenizer.padding_side = "right"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/895 [00:00<?, ?B/s]

configuration_decilm.py:   0%|          | 0.00/576 [00:00<?, ?B/s]

(…)sformers_v4_35_2__configuration_llama.py:   0%|          | 0.00/9.20k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Deci/DeciLM-7B:
- transformers_v4_35_2__configuration_llama.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

# Load dataset



In [ ]:
# !pip install apify_client

In [ ]:
# from apify_client import ApifyClient

# # Initialize the ApifyClient with your API token
# client = ApifyClient("your client") 

# # Prepare the Actor input
# run_input = {
#     "directUrls": ["https://www.instagram.com/ospreypacks/"], # your instagram page
#     "resultsType": "posts",
#     "resultsLimit": 4546,
#     "searchType": "hashtag",
#     "searchLimit": 1,
#     "addParentData": False,
# }

# # Run the Actor and wait for it to finish
# run = client.actor("your actor").call(run_input=run_input)

# # Fetch and print Actor results from the run's dataset (if there are any)
# for item in client.dataset(run["defaultDatasetId"]).iterate_items():
#     print(item)

In [ ]:
# client.dataset(run["defaultDatasetId"]).iterate_items()

In [ ]:
import pandas as pd

posts = pd.read_csv('/content/posts.csv')
posts.head()

,caption
0,Keep it close to the vest 🏃 \n\nEasy hydration...
1,"For a half century, @OspreyPacks has been driv..."
2,Who else is ready for hikes to look a bit more...
3,Configure a custom bikepacking system that fit...
4,"However your spring is looking, keep your day ..."


In [ ]:
posts.shape

(2270, 1)

In [ ]:
posts.dropna(axis=0, inplace=True)
posts.isna().sum()

caption    0
dtype: int64

In [ ]:
posts.shape

(2268, 1)

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(posts, test_size=0.1, shuffle=True)
train.shape, test.shape

((2041, 1), (227, 1))

In [ ]:
train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)

In [ ]:
from datasets import load_dataset

data_files = {"train": "train.csv", "test": "test.csv"}
dataset = load_dataset("csv", data_files=data_files)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['caption'],
        num_rows: 2041
    })
    test: Dataset({
        features: ['caption'],
        num_rows: 227
    })
})

# QLoRA Config

In [ ]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

# we set our lora config to be the same as qlora
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    #  The modules to apply the LoRA update matrices.
    target_modules = ["gate_proj", "down_proj", "up_proj"],
    task_type="CAUSAL_LM"
)

# Prepare model for peft

In [ ]:
if do_quantization:
    decilm = prepare_model_for_kbit_training(decilm)

decilm.enable_input_require_grads()
decilm = get_peft_model(decilm, lora_config)

# Training Args

In [ ]:
len(dataset['train'])

2041

In [ ]:
NUM_EPOCHS = 1
output_dir = '/content/fine-tuned-checkpoints'

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
        num_train_epochs = NUM_EPOCHS,
        output_dir=output_dir,
        evaluation_strategy="steps",
        do_eval=True,
        auto_find_batch_size=True,
        log_level="debug",
        optim="paged_adamw_32bit",
        save_steps=25,
        logging_steps=100,
        learning_rate=3e-4,
        weight_decay=0.01,
        max_steps=len(dataset['train']) * 5,
        warmup_steps=150,
        # if you're using a T4, or non-ampere supported chip comment out the below line.
        # bf16=True,
        # tf32=True,
        # gradient_checkpointing=True,
        # max_grad_norm=0.3, #from the paper
        # lr_scheduler_type="reduce_lr_on_plateau",
)

# Train



In [ ]:
import math

num_train_epochs = training_args.num_train_epochs
train_dataset_size = len(dataset['train'])
batch_size = training_args.per_device_train_batch_size * training_args.n_gpu if training_args.per_device_train_batch_size is not None else training_args.train_batch_size * training_args.n_gpu
total_steps = math.ceil((train_dataset_size / batch_size) * num_train_epochs)


In [ ]:
total_steps

256

In [ ]:
training_args.max_steps = total_steps

In [ ]:
trainer = SFTTrainer(
    model=decilm,
    args=training_args,
    peft_config=lora_config,
    tokenizer=tokenizer,
    dataset_text_field='caption',
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    max_seq_length=4096,
    dataset_num_proc=os.cpu_count(),
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/2041 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/227 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer.train()

Currently training with a batch size of: 8
***** Running training *****
  Num examples = 2,041
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 256
  Number of trainable parameters = 28,311,552
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
100,2.260200,1.934410
200,1.895300,1.863288


Saving model checkpoint to /content/fine-tuned-checkpoints/checkpoint-25
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Deci--DeciLM-7B/snapshots/c3c9f4226801dc0433f32aebffe0aac68ee2f051/config.json
Model config DeciLMConfig {
  "architectures": [
    "DeciLMForCausalLM"
  ],
  "attention_bias": false,
  "auto_map": {
    "AutoConfig": "Deci/DeciLM-7B--configuration_decilm.DeciLMConfig",
    "AutoModelForCausalLM": "Deci/DeciLM-7B--modeling_decilm.DeciLMForCausalLM"
  },
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "model_type": "deci",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "num_key_value_heads_per_layer": [
    4,
    4,
    4,
    4,
    4,
    2,
    2,
    2,
    2,
    2,
    4,
    2,
    2,
    2,
    2,
    2,
    2,
    2,
    2,
    2,
    1,
    1

TrainOutput(global_step=256, training_loss=2.027971923351288, metrics={'train_runtime': 5534.1452, 'train_samples_per_second': 0.37, 'train_steps_per_second': 0.046, 'total_flos': 1.7095250990481408e+16, 'train_loss': 2.027971923351288, 'epoch': 1.0})

In [ ]:
trainer.save_model()

Saving model checkpoint to /content/fine-tuned-checkpoints
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Deci--DeciLM-7B/snapshots/c3c9f4226801dc0433f32aebffe0aac68ee2f051/config.json
Model config DeciLMConfig {
  "architectures": [
    "DeciLMForCausalLM"
  ],
  "attention_bias": false,
  "auto_map": {
    "AutoConfig": "Deci/DeciLM-7B--configuration_decilm.DeciLMConfig",
    "AutoModelForCausalLM": "Deci/DeciLM-7B--modeling_decilm.DeciLMForCausalLM"
  },
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "model_type": "deci",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "num_key_value_heads_per_layer": [
    4,
    4,
    4,
    4,
    4,
    2,
    2,
    2,
    2,
    2,
    4,
    2,
    2,
    2,
    2,
    2,
    2,
    2,
    2,
    2,
    1,
    1,
    1,
    1

In [ ]:
trainer.save_model("tuned_decilm-7b")

Saving model checkpoint to tuned_decilm-7b
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Deci--DeciLM-7B/snapshots/c3c9f4226801dc0433f32aebffe0aac68ee2f051/config.json
Model config DeciLMConfig {
  "architectures": [
    "DeciLMForCausalLM"
  ],
  "attention_bias": false,
  "auto_map": {
    "AutoConfig": "Deci/DeciLM-7B--configuration_decilm.DeciLMConfig",
    "AutoModelForCausalLM": "Deci/DeciLM-7B--modeling_decilm.DeciLMForCausalLM"
  },
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "model_type": "deci",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "num_key_value_heads_per_layer": [
    4,
    4,
    4,
    4,
    4,
    2,
    2,
    2,
    2,
    2,
    4,
    2,
    2,
    2,
    2,
    2,
    2,
    2,
    2,
    2,
    1,
    1,
    1,
    1,
    1,
    1,


# Merge adapter to base model




In [ ]:
fine_tuned_model = decilm.merge_and_unload()
fine_tuned_model

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:325: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


DeciLMForCausalLM(
  (model): DeciLMModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-4): 5 x DeciLMDecoderLayer(
        (self_attn): DeciLMAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=512, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=512, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaDynamicNTKScalingRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
      (5

In [ ]:
from peft import AutoPeftModelForCausalLM
from functools import partial

AutoTokenizer.from_pretrained = partial(AutoTokenizer.from_pretrained, trust_remote_code=True)

instruction_tuned_model = AutoPeftModelForCausalLM.from_pretrained(
    training_args.output_dir,
    torch_dtype=torch.bfloat16,
    # offload_folder="offload_dir/", offload_state_dict = True,
    device_map = 'auto',
    trust_remote_code=True,
)

merged_model = instruction_tuned_model.merge_and_unload()

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Deci--DeciLM-7B/snapshots/c3c9f4226801dc0433f32aebffe0aac68ee2f051/config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Deci--DeciLM-7B/snapshots/c3c9f4226801dc0433f32aebffe0aac68ee2f051/config.json
Model config DeciLMConfig {
  "_name_or_path": "Deci/DeciLM-7B",
  "architectures": [
    "DeciLMForCausalLM"
  ],
  "attention_bias": false,
  "auto_map": {
    "AutoConfig": "Deci/DeciLM-7B--configuration_decilm.DeciLMConfig",
    "AutoModelForCausalLM": "Deci/DeciLM-7B--modeling_decilm.DeciLMForCausalLM"
  },
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "model_type": "deci",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "num_key_value_heads_per_layer": [
    4,
    4,
    4

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing DeciLMForCausalLM.

All the weights of DeciLMForCausalLM were initialized from the model checkpoint at Deci/DeciLM-7B.
If your task is similar to the task the model of the checkpoint was trained on, you can already use DeciLMForCausalLM for predictions without further training.
Generation config file not found, using a generation config created from the model config.
loading file tokenizer.model
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 32000. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multip

NotImplementedError: Cannot copy out of meta tensor; no data!

In [ ]:
from transformers import pipeline

generation_kwargs = {
    "max_new_tokens": 100,
    "early_stopping": True,
    "num_beams": 5,
    "temperature" : 0.001,
    "do_sample":True,
    "no_repeat_ngram_size": 3,
    "repetition_penalty" : 1.5,
    "renormalize_logits": True,
    "top_p" : 0.95
}

decilm_tuned_pipeline = pipeline(
    "text-generation",
    model=fine_tuned_model,
    tokenizer=tokenizer,
    **generation_kwargs
)

In [ ]:
outputs = decilm_tuned_pipeline("In a shocking finding, scientists discovered a herd of unicorns living in")
print(outputs[0]["generated_text"])

In [ ]:
decilm_tuned_pipeline('create a new post about the "Adventure" 25-liter backpack for $200, which is great for climbers', return_full_text=False)[0]['generated_text']

In [ ]:
decilm_tuned_pipeline('create a new post about our new sport bag for travalers', return_full_text=False)[0]['generated_text']

In [ ]:
decilm_tuned_pipeline('write me a post aboutgiveaway of 3 bags of the new collection', return_full_text=False)[0]['generated_text']

In [ ]:
# from huggingface_hub import notebook_login
# notebook_login()

In [ ]:
# fine_tuned_model.push_to_hub("Post_Generaton_Fine_tuned_DeciLM_7B",)